In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=BblV6AQsd2s&ab_channel=RiseAgainstVEVO"

### Setting up the model
Define the LLM model to use

In [15]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

model.invoke("Are we in a simulation?")

AIMessage(content='There is currently no definitive evidence to prove or disprove the idea that we are living in a simulation. Some scientists and philosophers have proposed the concept of a simulated reality, similar to the simulation hypothesis, where our universe and all of reality as we know it is being generated by a super-advanced computer program. However, this is still a speculative theory and is not widely accepted in the scientific community. Ultimately, the question of whether or not we are in a simulation remains an open and debated topic.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

### Using the magic of Chains :)

In [16]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Chain example
chain = model | parser

chain.invoke("Are we in a simulation?")

'There is currently no definitive evidence to prove that we are living in a simulation. The idea that we could be living in a simulation is a philosophical concept that has been debated for many years. Some scientists and philosophers argue that it is a possibility, while others believe it is unlikely. Ultimately, the question of whether or not we are in a simulation remains unanswered.'

### Using prompt templates

In [17]:
from langchain_core.prompts import ChatPromptTemplate

template = """
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: {context}

  Question: {question}
  """

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="The sky is blue.", question="What color is the sky?"))

print("Did it work?")

chain = prompt | model | parser
chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?"
})

Human: 
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: The sky is blue.

  Question: What color is the sky?
  
Did it work?


'No, you have an appointment at 3pm tomorrow.'

### Combining Chains

In [18]:
from operator import itemgetter

translation_template = """
  Translate the 

  answer: {answer}

  to

  language: {language}
  """

translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
  {"answer": chain, "language": itemgetter("language") } | translation_prompt | model | parser
)

translation_chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?",
  "language": "es"
})


'Respuesta: No, no puedes ir a almorzar mañana a las 3:00 PM porque tienes una cita programada para esa hora.'

### Transcribing the YouTube Video

In [31]:
import tempfile
import whisper
from pytube import YouTube

if not os.path.exists("transcription.txt"):
  youtube = YouTube(YOUTUBE_VIDEO)
  audio = youtube.streams.filter(only_audio=True).first()

  whisper_model = whisper.load_model("base")

  script_dir = os.path.expanduser('~/Documents/tmp')

  with tempfile.TemporaryDirectory() as temp_dir:
    print(f"** Downloading {audio.title} to {temp_dir}")
    # Download the audio file with this name transcription_audio.mp4
    file = audio.download(temp_dir, filename="transcription_audio.mp4")
    
    # Print the file path
    print(file)
    # Validate if the file exists
    print(os.path.exists(file))
    # Print the file name
    print(os.path.basename(file))

    transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()
    
    with open("transcription.txt", "w") as f:
     f.write(transcription)

    transcription

** Downloading Swing Life Away Closed Captioned to /var/folders/c1/ykm5wz554pq8yml54fj3gtkm0000gn/T/tmpnz90l_up
/var/folders/c1/ykm5wz554pq8yml54fj3gtkm0000gn/T/tmpnz90l_up/transcription_audio.mp4
True
transcription_audio.mp4
